## MultiClassifier Neural Network

![](https://github.com/MhmdSyd/needed_image/blob/main/mnist2.png?raw=true)

**The notebook is structured in the following way:**

> Data Summary.

> Data Preparation.

> Modeling.

> Model Analysis.


**Import Necessary Librariess**

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage import color
from skimage import measure
from skimage.filters import try_all_threshold
from skimage.filters import threshold_otsu
from skimage.filters import threshold_local
import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, AveragePooling2D, MaxPool2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

**Read Data and Split to X and Y**

In [ ]:
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
df_test = pd.read_csv('../input/digit-recognizer/test.csv')

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<span style='padding: 10px; font-family:Helvetica; font-size:30px' > Data Summary </span>
</div>

- **Size of the dataset.**

In [ ]:
print("Train Data Size: ",df_train.shape)
print("Test Data Size: ",df_test.shape)

## Value Count Of Target

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Numbers vs Count in Label', fontsize=18, y=1.02)

ax = sns.barplot(x=[*range(10)],
                 y=df_train.label.value_counts().sort_index(),
                 color='#255db8')

for p in ax.patches:
        ax.annotate('{}'.format(int(p.get_height())),
                    (p.get_x()+0.2, p.get_height()+50))

plt.show()

## Display Sample of Train Images

In [ ]:
plt.style.use('seaborn')
fig, ax = plt.subplots(1,6,figsize=(10,10))
fig.suptitle('Sample of Train Images', y=0.6, fontsize=18, color="#910356")
fig.tight_layout(pad=0.0)

for col in range(6):

    im_0 = np.array(df_train.drop("label", axis=1).iloc[col]).reshape(28,28)
    
    ax[col].imshow(im_0, cmap="gray")
    
    ax[col].axis("off")

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Helvetica;
           letter-spacing:0.5px">

<span style='padding: 10px; font-family:Helvetica; font-size:30px' > Data Preparation </span>
</div>

## Split Data

In [ ]:
y_train = df_train['label']
X_train = df_train.drop('label', axis = 1)

X_test = np.array(df_test)

- **Reshape Data**

In [ ]:
X_train = np.array(X_train).reshape((-1,28,28,1))
X_train.shape

In [ ]:
X_test = np.array(X_test).reshape((-1,28,28,1))
X_test.shape

In [ ]:
y_train = to_categorical(y_train, num_classes = 10)
y_train.shape

- **Split Train Dataset To Train And Validation Data**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size=0.25,
                                                  random_state=1)


<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Helvetica;
           letter-spacing:0.5px">

<span style='padding: 10px; font-family:Helvetica; font-size:30px' > Modeling </span>
</div>

- **Set Kernel Matrix Size**

In [ ]:
kernel_ = (5,5)

## Create Model and Add Layers

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

- **Generate Image Train Data**

In [ ]:
aug = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1)

gen_train = aug.flow(X_train, y_train, batch_size=64)

gen_val = aug.flow(X_val, y_val, batch_size=64)

## Compile Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import tensorflow as tf

checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",
                                                 monitor='val_accuracy',
                                                 verbose=1,
                                                 save_best_only=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                  factor=0.5,
                                                  patience=3,
                                                  min_lr=0.00005,
                                                  verbose=1)

early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

## Train Model

In [ ]:
training = model.fit(gen_train, epochs=50, batch_size=128,
                    validation_data=gen_val,
                    callbacks = [checkpoint, reduce_lr], verbose = 1)

In [ ]:
model.load_weights("weights.hdf5")

## Predict Test Target

In [ ]:
y_test = model.predict(X_test)
y_pred = np.argmax(y_test, axis=1)

- **Save Prediction Target**

In [ ]:
output_csv = {"ImageId":[*range(1,1+len(y_pred))], "Label":y_pred}
Y_pre = pd.DataFrame(output_csv)
Y_pre.set_index("ImageId", drop=True, append=False, inplace=True)
Y_pre.to_csv("/kaggle/working/submission.csv")


<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Helvetica;
           letter-spacing:0.5px">

<span style='padding: 10px; font-family:Helvetica; font-size:30px' > Model Analysis </span>
</div>

- **Loss With epochs for Train and Validation Data**

In [ ]:
# plot loss function for train and validation.
plt.title('loss and validation loss with epochs', 
          fontsize=16)

plt.plot(training.history['loss'],
         marker='o',
         color="green",
         label="loss")

plt.plot(training.history['val_loss'],
         marker='o',
         color="orange", 
         label="val_loss")

plt.legend()
plt.show()

## Accuracy With epochs for Train and Validation Data

In [ ]:
# plot accuracy for train and validation data.
plt.title('accuracy and validation accuracy with epochs',
          fontsize=16)

plt.plot(training.history['accuracy'],
         marker='o',
         color="green",
         label="accuracy")

plt.plot(training.history['val_accuracy'],
         marker='o',
         color="orange",
         label="val_accuracy")

plt.legend()
plt.show()

## Test Sample Image With Its Prediction

In [ ]:
plt.style.use('seaborn')

fig, ax = plt.subplots(2,5,figsize=(12,5))
fig.suptitle('Sample of Test Images and Prediction', y=1.2, fontsize=18, color="#910356")
fig.tight_layout(pad=-2)

for col in range(5):
    
    im_0 = np.array(df_test.iloc[col]).reshape(28,28)
    
    ax[0][col].imshow(im_0, cmap="gray")
    ax[0][col].set_title('label: '+str(y_pred[col]))
    ax[0][col].axis("off")
    
    im_0 = np.array(df_test.iloc[col+6]).reshape(28,28)
    
    ax[1][col].imshow(im_0, cmap="gray")
    ax[1][col].set_title('label: '+str(y_pred[col+6]))
    ax[1][col].axis("off")

### Thanks For Read My NoteBook :)